#  <font color=darkcyan> MCMC : Adaptative Tuning of Hamiltonian Monte Carlo Within Sequential Monte Carlo

<div class="alert alert-success">
    Authors
     <ul>
         <li>Hugues GALLIER
         <li>Guillaume HOFMANN
         <li>Christos KATSOULAKIS
    </ul>
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""""""""""""""""
Required packages
"""""""""""""""""
import autograd.numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import pandas as pd
import pickle
import statsmodels.formula.api as smf

from autograd.numpy.linalg import inv
from autograd import grad
from dataclasses import dataclass
from scipy import stats, optimize
from types import FunctionType

-------------

# I. Context

## <div style="padding-left: 15px;"> i. Introduction

The article chosen for this project is the following : *Adaptive tuning of Hamiltonian Monte Carlo within sequential Monte Carlo* [[1]](#bib). This research paper written by Buchholz et al. was recently published in July 2020 in the journal *Bayesian Analysis* of ISBA which is [The International Society for Bayesian Analysis](https://bayesian.org/).
    
By starting to sample particles from an initial distribution $\pi_0$, Sequential Monte Carlo (SMC) samplers enable to approximate a target distribution $\pi$ by iterating through a sequence of distributions $\pi_t$. This approach is a powerful alternative to MCMC in Bayesian computation for several reasons : SMC samplers are robust to multimodality, they can benefit from parallel computing in the way of generating and moving the particles over the iterations, and they allow the estimation of the normalizing constants.

The propagation of the particles over the iterations might be the most challenging part of SMC and commonly relies on MCMC kernels. Tuning the MCMC kernels parameters has been of interest over the past few years and the authors of the paper decided to focus on HMC (Hamiltonian Monte Carlo) kernels. Indeed, originally developed in Physics, HMC has become a standard tool in MCMC, especially because of its better mixing in high dimensional problems. 

The tuning of Markov kernels within SMC samplers can be linked to the tuning of MCMC kernels in general. In this paper, the authors compares methods for the automatic tuning of HMC kernels within SMC. They firstly rely on the work of Fearnhead and Taylor (2013) and adapt their tuning procedure to the HMC kernel and, then, introduce an alternative approach based on a pre-tuning phase at each intermediate step.

## <div style="padding-left: 15px;"> ii. Sequential Monte Carlo
### <div style="padding-left: 30px;"> a) Classical settings of Sequential Monte Carlo

A classical problem that arises when doing Bayesian inference is to estimate the following unknown quantity : 
$$E_{\pi}(\phi) = \int \pi(dx)\phi(x)$$
where $\phi$ is a measurable function, and where $\pi$ is a posterior distribution such that :$$\pi(x) = \pi(x|y) = \frac{p(x)l(y|x)}{Z}$$ with $p$ the prior and $l$ the likelihood.

The idea of Sequential Monte Carlo (SMC) is to introduce some intermediate distributions $\pi_{0}, ..., \pi_{t}, ..., \pi_{T}$ such that it is easy to sample from $\pi_{0}$, with $\pi_{T} = \pi$, and such that we can use a Markov Kernel $\mathcal{K}_{t}$ to simulate under $\pi_{t+1}$ given particles approximating $\pi_{t}$. A classical approach to define those intermediate densities is called tempering (other procedures to choose the intermediate distributions exist, but we will focus, as the article does, on tempering) and consists in taking :
$\forall t \in \{1, ..., T\}$, $$ \pi_{t}(x) \propto p(x) l(y|x)^{\lambda_{t}} $$  
where $0 = \lambda_{0} < \lambda_{1} < ... < \lambda_{T} = 1$

The idea of the SMC algorithm described in this article is to do an exploration step at each time t from $\pi_{t}$ to the same distribution, then do a reweighting step, and finally do a resampling step. The following describe more precisely this algorithm.

Suppose we have a set of particles $X_{t}$ targeting $\pi_{t}$ (all particles are equally weighted). The exploration step is done by taking a Markov Kernel $\mathcal{K}_t$ letting $\pi_{t}$ invariant, and drawing : $$\tilde{X}_{t} \sim \mathcal{K}_t(X_{t}, dx)$$ 

The obtained particles still target $\pi_{t}$, and are still equally weighted (see calculations in the lecture notes).


A reweighting step is then done using the particles $(\tilde{X}_{t})$. Using the (known) unnormalized densities $\gamma_{t}$, $t = 1, ..., T$ defined by : 

$$\gamma_{t} (x) = p(x)l(y|t)^{\lambda_{t}}$$

We obtain that the particles $(\tilde{X}_{t}, \frac{\gamma_{t+1}}{\gamma_{t}})$ (where the second component is the weight associated to the particle) are now targeting $\pi_{t+1}$ !  

The following subsection gives more details about Importance Sampling, which is the underlying idea behind a reweighting step.

Finally, a resampling step is performed, in order to obtain particles that are equally weighted. As there will be possible duplicates amongst the particles, the exploration state will be of importance in order to recover some variability.

Before presenting Importance Sampling, we would like to show in the following cells some core classes that will be of use in the following. Those classes are:
- A dataclass ExperimentParameters, which is intented to provide the SMC sampler with all information needed for a new experiment (mainly : the log prior, the log likelihood, the initial distribution to sampler from, the dimension $d$, the number of particles $N$, and other attributes that can be of use).
- A class SMC Sampler, which is the parent class of all subsequent SMC samplers with specific kernels.

In [ ]:
@dataclass
class ExperimentParameters:
    """Store experiment parameters."""

    log_prior: FunctionType  # p(.)
    log_likelihood: FunctionType  # l(y|.)
    pi0: stats._distn_infrastructure.rv_frozen  # distribution to sample from
    grad_log_gamma_t: FunctionType = None
    N: int = 1000
    d: int = 10
    alpha: float = 0.5  # for ESS-based rule of choosing lambda
    normalizing_constant: float = 1

    def __post_init__(self):
        def _grad_log_gamma_t(lambda_t, X):
            gradl = grad(lambda x: self.log_gamma_t(lambda_t, np.array([x]))[0])
            result = np.array([gradl(X[i]) for i in range(len(X))])
            return result
        self.grad_log_gamma_t = self.grad_log_gamma_t or \
            _grad_log_gamma_t

    def log_gamma_t(self, lambda_t, X):
        return self.log_prior(X) + self.log_likelihood(X) * lambda_t

    def log_ratio_gamma(self, lambda_t, lambda_t_prime, X):
        """Performs ratio of gamma function lambda_t_prime / gamma 
        function lambda_t."""
        return self.log_likelihood(X) * (lambda_t_prime - lambda_t)

In [ ]:
class SMCSampler:
    """Implement Sequential Monte Carlo methods using."""

    def __init__(self, parameters : ExperimentParameters):

        assert isinstance(parameters, ExperimentParameters), \
            "Please use proper dataclass."

        self.param = parameters
        self.X = None  # liste
        self.weights = np.zeros(parameters.N)
        self.constants_ratio = []
        self.lambdas = [0]
        
    def run(self):  # algo 1
        t = 1
        lambda_t = 0
        while lambda_t < 1:

            if t == 1:
                X = self.param.pi0.rvs(size=self.param.N)

            else:
                X = self.move_through_kernel(X, lambda_t)       # algo 3, 5, 6

            lambda_tp1 = self.choose_next_lambda(X, lambda_t)
            log_weights = self.compute_log_weights(X, lambda_t, lambda_tp1)
            lambda_t = lambda_tp1

            # resampling
            weights = np.exp(log_weights)
            weights = weights / weights.sum()
            X = X[np.random.choice(np.arange(len(X)), size=len(X), p=weights)]
            
            self.X = X  
            self.weights = weights
            self.constants_ratio += [np.mean(weights)]
            self.lambdas += [lambda_tp1]

            t += 1

    def move_through_kernel(self, X, lambda_t):
        raise NotImplementedError("Do not try to run this parent class.")

    def compute_log_weights(self, X, lambda_t, lambda_tp1):
        # cut weights arbitratitly to avoid overflow
        return np.minimum(100, self.param.log_ratio_gamma(lambda_t, lambda_tp1, X))
 
    def choose_next_lambda(self, X, lambda_t):  
        """Find next exponent according to algorithm 2 (Beskos et al. 2016)."""

        def ESS_alphaN(lbda):
            """Compute ESS in our special case for given lambda_tp1.
            Substract max of weights to avoid overflow.
            """
            log_weights = self.compute_log_weights(X, lambda_t, lbda)
            max_log_weights = np.max(log_weights)
            log_weights -= max_log_weights
            ESS = np.sum(np.exp(log_weights)) ** 2 / np.sum(np.exp(2*log_weights))
            return ESS - self.param.alpha * self.param.N

        if ESS_alphaN(1) >= 0:
            return 1

        lambda_tp1 = optimize.root_scalar(ESS_alphaN, bracket=[lambda_t, 1],
                                          method='brentq', xtol=1e-2)
        return lambda_tp1.root

### <div style="padding-left: 30px;"> b) Importance Sampling

Importance Sampling (IS) is a method allowing to approximate a density $g$ (the target distribution) using a simpler density $f$ from which we can draw some particles. 
The procedure is as follow : suppose that we have $\psi$ a measurable function, and that we we want to estimate $E_{X \sim g}(\psi(X))$. Then, one can rewrite this quantity in the following way :

$$ \int \psi(x) g(dx) = \int \psi(x) \frac{g(x)}{f(x)} f(dx)$$

Note that we suppose that $f$ and $g$ are both dominated by the same measure $\lambda$, that $f \ne 0$ and that $f(dx) = f(x) \lambda(dx)$ by simplicity of notation.

This gives us that to estimate $E_{X \sim g}(\psi(X))$, one can take particles simulated under $f$, and weight them by the ratio : 

$$ \omega(x) = \frac{g(x)}{f(x)} $$ 

Importance sampling is also useful when we know $f$ and $g$ only up to an unknown constant. In fact, one can then use _self normalized Importance Sampling_ where the weights are individualy divided by the sum of all weights. 

### <div style="padding-left: 30px;"> c) Choice of the Markov Kernel <a name="choicekernel">

How can we choose $\mathcal{K}_t$ that gives good result (in a sense to be defined later) ?

A natural choice is to take an MCMC Kernel, for example a MALA Kernel. We have implemented this MCMC Kernel, and you can find a complete SMC algorithm implementing MALA moves thereafter.

Of course, the tuning of this Markov Kernel is crucial to the performance of the obtained algorithm. This tuning can be done before run-time, or "on the fly". A great advantage of doing this during run-time is that we can use the particles at hand to tune it, and this information can be a key towards a much better performance.

An adaptative method to tune a Independent Metropolis Hastings Kernel within SMC was proposed by N. Chopin [[2]](#bib) in 2002. The idea relies on the fact that $\pi_{t+1}$ is quite similar to $\pi_{t}$, and thus that we can use the particles at hand (and in particular their empirical covariance) in order to propose in an efficient manner. The chosen proposal is simply a Gaussian with expectation $\widehat{E}$ and covariance matrix $\widehat{V}$ as defined below : 

$$ \widehat{E} = \widehat{E}_{\pi_{t}}(X_{t}) , \quad \widehat{V} = \widehat{Var}_{\pi_{t}}(X_{t}) $$

Note that this independence sampler results in particle $X_{t}^{i}$ depending on $X_{t+1}^{i}$ only through the acceptance ratio (and through the estimation of $\hat{E}$ and $\hat{V}$). This is quite different from the common Independence Sampling moves, where the Gaussian is centered around $X_{t}^{i}$, and not around the mean of all the particles at time $t$. The advantage of this technique is that the obtained distribution is closed to the next distribution, so that the acceptance rate of the new particles gives a good approximation of how precise our algorithm is. Otherwise, if $X_{t+1}^{i}$  was drawn around $X_{t}^{i}$, one could arbitrarily increase the acceptance ratio by decreasing the variance of the proposals. The algorithm described here is also proposed in this notebook.

Another adaptative tuning approach of MCMC kernels within SMC was developed in Fearnhead and Taylor [[3]](#bib) in 2013. It will developed later.

You can see in the following cells our implementation of two SMC Sampler with different Markov Kernels :
- MALASMCSampler implements an SMC sampler with MALA moves
- TunedISMC implements an SMC sampler described in [[2]](#bib) N. Chopin (2001), where the kernel at time $t$ is an independence sampler where the proposal is a Normal distribution of expectation and variance equal respectively to the empirical mean and empirical variance of the particles at time $t$.

In [ ]:
class MALASMCSampler(SMCSampler):
    """Implement basic SMC with MALA Kernel."""

    def __init__(self, parameters: ExperimentParameters, sigma = 0.2):
        super().__init__(parameters)
        self.sigma = sigma
        
    def move_through_kernel(self, X, lambda_t):
        """Non-iterated version : single move for each x in X."""

        N = self.param.N
        d = self.param.d
        self.sigma = d ** (-1/3)
        
        eps = stats.multivariate_normal(np.zeros(d), np.eye(d)).rvs(N)

        mean_old = X + 0.5 * self.sigma * self.param.grad_log_gamma_t(lambda_t, X)
        Z = mean_old + self.sigma * eps
        mean_new = Z + 0.5 * self.sigma * self.param.grad_log_gamma_t(lambda_t, Z)

        # performs np.dot(Z[i] - mean_old, Z[i] - mean_old) for all i
        q_x_z = - 0.5 * np.einsum('ij,ij->i', Z - mean_old, Z - mean_old) / (self.sigma ** 2) 
        q_z_x = - 0.5 * np.einsum('ij,ij->i', X - mean_new, X - mean_new) / (self.sigma ** 2)
        prob = self.param.log_gamma_t(lambda_t, Z) + q_z_x \
            - (q_x_z + self.param.log_gamma_t(lambda_t, X))
        
        # Keep new moves or not
        log_acc = np.log(np.random.rand(len(X)))
        keep_it = log_acc < prob
        keep_it = keep_it[:, np.newaxis]
        
        return Z * keep_it + X * (1 - keep_it)

In [ ]:
class TunedISMC(SMCSampler):
    """Implement N. Chopin (2001) approach to construct a tuned Independence
    Sampler Kernel."""

    def __init__(self, parameters: ExperimentParameters):
        super().__init__(parameters)
    
    def move_through_kernel(self, X, lambda_t):
        """Single move for each x in X using Independence Sampler properly 
        calibrated."""

        # First calculate mean and covariance
        E = np.mean(X, axis=0)
        assert E.shape == (self.param.d,), "Wrong shape for particles mean."
        cov = np.cov(X, rowvar=False, bias=True)
        V = np.diag(np.diag(cov))
        inv_V = np.diag(1/np.diag(cov))
        assert V.shape == (self.param.d, self.param.d), \
            "Wrong shape for particles variance-covariance matrix."
        
        # Simulate new observations
        g = stats.multivariate_normal(E, V)
        Z = g.rvs(self.param.N)

        # Compute probabilities
        log_pdf_x = -0.5 * np.einsum('ij,ij->i', X - E, 
                                     np.dot(inv_V, (X - E).T).T)
        log_pdf_z = -0.5 * np.einsum('ij,ij->i', Z - E, 
                                     np.dot(inv_V, (Z - E).T).T)
        
        log_proba = log_pdf_x - log_pdf_z + self.param.log_gamma_t(lambda_t, Z) \
            - self.param.log_gamma_t(lambda_t, X)
            

        # Keep new moves or not
        log_acc = np.log(np.random.rand(len(X)))
        keep_it = log_acc < log_proba
        keep_it = keep_it[:, np.newaxis]

        return Z * keep_it + X * (1 - keep_it)

## <div style="padding-left: 15px;"> iii. Hamiltonian Monte Carlo

Hamiltonian Monte Carlo (HMC) constructs, on an extended space, a Markov Chain that admits an invariant probability measure $\Pi$ whose marginal distribution is the target distribution. An auxiliary density is introduced (a multivariate normal in most applications) to extend the target density. In the context of the article the distribution $\Pi$ is :  

<center>
    $$\Pi(x,p) \propto \exp(\mathcal{L}(x)-\frac{1}{2} p^{T}M^{-1}p) \propto \exp(-H(x,p))$$ <br>
    with the Hamiltonian $H(x,p)=-\mathcal{L}(x)+\frac{1}{2} p^{T}M^{-1}p$
</center>

This density is the product of two independent densities : the first term is the target density $\gamma(x)= \exp(\mathcal{L}(x))$ where $\mathcal{L}(x)$ is the unnormalized log density of the random variable of interest, the second term is the normal density $\mathcal{N}(0,M)$.

It is possible to construct a Metropolis Hasting algorithm with deterministic moves that targets $\Pi$. An assumption is needed for the Metropolis Hasting kernel to be $\Pi$ reversible : the deterministic function is an involution, that means it is a function that is its own reverse.

For any involution, the acceptance probability can be equal to one if the involution stays on the same level set (the moves according to the deterministic function does not change the value of $\Pi$) and if the involution is volume preserving (the Jacobian determinant of the deterministic function is equal to one).

Moves that stay on the same level set do not change the value of $\Pi$ and consequently do not change the value of the Hamiltonian $H$. Thus, we have the following equations to obtain deterministic moves that stay on the same level:    
    
$$\frac{dx}{dt} = \frac{\partial H(x,q)}{\partial p} = M^{-1}p$$ 
$$\frac{dp}{dt} = -\frac{\partial H(x,q)}{\partial x} = \nabla_{x} \mathcal{L}(x) $$

This system is called Hamiltonian dynamics. In addition of staying on the same level sets, the moves proposed by the Hamiltonian dynamics are volume preserving. However it is not an involution but it can be resolved by using the flip operator trick on the second component after the move (the flip operator keeps intact the same level sets and volume preserving properties).

The formulation of Hamiltonian can be seen from a physical perspective : the Hamiltonian is the sum of the potential energy at the position $x$ and the kinetic energy of the momentum $p$ with mass matrix $M$. 

In this framework, the Hamiltonian dynamics describes the motion of a particle evolving in a potential. The solution induces a flow that describes the evolution of a system from an initial position and momentum. 

In general, we cannot have the exact solution of the Hamiltonian dynamics so numerical integration methods are used. One of the most applied is the Leapfrog integrator:

$$p_{k+\epsilon/2} = p_{k} + (\epsilon/2) \nabla_{x} \mathcal{L}(x_{k})$$
$$x_{k+\epsilon} = x_{k} + \epsilon M^{-1}p_{k+\epsilon/2}$$
$$p_{k+\epsilon} = p_{k+\epsilon/2} + (\epsilon/2) \nabla_{x} \mathcal{L}(x_{k+\epsilon})$$

$L$ steps are made in order to move from an instant $t$ to the instant $t+L\epsilon$ where $\epsilon$ is the step size of the Leapfrog integrator. 

The Leapfrog update is volume preserving (each operation freezes one component and update the other with an additive term) but the variation of energy is not null therefore it is not level-set invariant.

HMC and Leapfrog integrator introduce several parameters that are important to tune properly in order to achieve good performance.

First, there is the covariance matrix M that affects the sampling of $p$ and the update of $x$ during the Leapfrog step.

Then, the choice of $\epsilon$ and $L$ are challenging and important to obtain stable trajectories in a reasonable time. Indeed, the error of energy and the error of position and momentum induced by the Leapfrog integrator increase in $\epsilon^2$ times a constant. In the case of the error of position and momentum, the constant increases with L.

As a result, if $\epsilon$ is too large, the simulation will be inaccurate and the acceptance rate will be low because the variation in energy is large. If $\epsilon$ is too small, the exploration will be small. To counterbalance small trajectories, $L$ should be large that increases the computation time. If $L$ is too small, the successive samples are closed and will behave as a random walk.

We provide here:
- A implementation of an SMC sampler with non tuned HMC moves.

In [ ]:
class NonTunedHSMCSampler(SMCSampler):
    """Implement basic SMC with non tuned SMC kernel."""

    def __init__(self, parameters: ExperimentParameters, epsilon=0.05, L=5):
        super().__init__(parameters)
        self.epsilon = epsilon
        self.L = L
    
    def _leapfrog_integrator(self, X_init, p_init, lambda_t):

        _X = X_init.copy()
        _p = p_init.copy()
        
        for l in range(self.L):

            _p += (self.epsilon / 2) * self.param.grad_log_gamma_t(lambda_t, _X)
            _X += self.epsilon * _p
            _p += (self.epsilon / 2) * self.param.grad_log_gamma_t(lambda_t, _X)

        return _X, _p

    def move_through_kernel(self, X, lambda_t):
        """Non-iterated version : single move for each x in X."""

        N = self.param.N
        d = self.param.d
        
        p = stats.multivariate_normal(np.zeros(d), np.eye(self.param.d)).rvs(N)
        Z, p_new = self._leapfrog_integrator(X, p, lambda_t)

        log_proba = self.param.log_gamma_t(lambda_t, X) - \
            self.param.log_gamma_t(lambda_t, Z) + (np.einsum('ij,ij->i', p_new, p_new) \
                -  np.einsum('ij,ij->i', p, p)) / 2

        # Keep new moves or not
        log_acc = np.log(np.random.rand(len(X)))
        keep_it = log_acc < log_proba
        keep_it = keep_it[:, np.newaxis]

        return Z * keep_it + X * (1 - keep_it)

# II. Research Issue
## <div style="padding-left: 15px;"> i. Matrix Choice

As already explained above, one of the major challenges in HMC is to find the appropriate Mass Matrix $M$ used to sample the Momentum, and used again in the Leapfrog integrator.

The approach explained in [I.ii.c](#choicekernel), which consists in taking an estimation of the covariance matrix of the current particles to estimate M, can be used again. This time nonetheless, the approach of the article is to take only the diagonal of this matrix, which results in the approach being applicable in high dimensions, and inverting it. In fact, as the leapfrog intergrator tends to move faster in the direction given by the inverse of $M$, one would want to take

$$ M = diag\left(\widehat{Var}_{\pi_{t}}(X_{t})\right)^{-1} $$

## <div style="padding-left: 15px;"> ii. Fearnhead and Taylor's tuning adaptation to HMC <a name="FT">


Several techniques exist in order to choose the optimal step size $\epsilon$ and number of steps $L$. However, the approach that is mostly developed throughout this article is the maximization of the expected squared
jumping distance (ESJD), which in one dimension can be defined as following:  
<!-- <center><br> -->
$$ \mathrm{ESJD} = \mathbb{E}[||x_s - x_{s-1}||_2^2] = 2(1-\rho_1)\mathrm{Var}_\pi [x]$$ <br>
where $\rho_1$ is the first order autocorrelation.
<!-- </center><br> -->
The intuition is that in dimension $d$, maximizing the ESJD in euclidean norm is equivalent to minimizing the correlation of the $d$ dimensional process.

This setting is close to the one developed by Fearnhead and Taylor [[3]](#bib) in 2013. At the time, Fearnhead and Taylor considered ESJD optimization at each time $t$ as a way to deduce Adaptative Sequential Monte Carlo (ASMC) samplers, in which hyperparameters $h$ of transition kernels $\mathcal{K}^h_t$ were tuned at each time $t$.  As this remains rather comprehensive, it seems like a good idea to adapt it to Hamiltonian Monte Carlo transition kernels.

Fearnhead and Taylor consider the following ESJD criterion:

$$g_t(h) = \int \pi_{t-1}(x_{t-1})\mathcal{K}^h_t(x_{t-1},x_t)||x_{t-1}-x_t||^2_M dx_{t-1} dx_t$$

where $||x-y||^2_M = (x-y)^tM^{-1}(x-y)$ is the Mahalanobis distance with respect to $M$. Here, we set $M = \mathbf{M}_{t-1}$. For each particle $i$, this quantity can be estimated by the following Rao-Blackwellized estimator, divided by $L$ to take computations cost into account:

$$\tilde{\Lambda}(\tilde{x}^i_{t-1},\hat{x}^i_t)= \frac{||\tilde{x}^i_{t-1}-\hat{x}^i_{t}||^2_M}{L} \times \mathrm{min}(1,\mathrm{exp}[\Delta E^i_t])$$

where $\hat{x}^i_{t}$ is the proposed position based on the Hamiltonian dynamics before the Metropolis-Hasting step.

This estimation can then be used as a performance metric for each $h^i$. From there, the idea of Fearnhead and Taylor is that, at each step $t$, pairs $h^i_t = (\epsilon^i_t,L^i_t)$ should be randomly drawn according to their performance during the previous step $t-1$ so that good performing sets of hyperparameters have better chances to be chosen.
In order to avoid pathological behaviors, it is still important to randomize a little bit the choice of the $h^i_t$ pairs. To that extent, a perturbation kernel $R$ is introduced.

$$R(h;h^i_{t-1})=\mathcal{TN}(\epsilon; \epsilon^i_{t-1} , 0.015^2) \otimes \{ \frac{1}{3}\mathbf{1}_{\{ L^i_{t-1} -1 \}} (L) + \frac{1}{3}\mathbf{1}_{\{ L^i_{t-1} \}} (L) + \frac{1}{3}\mathbf{1}_{\{ L^i_{t-1} +1 \}} (L) \}$$

where $\mathcal{TN}$ denotes a normal distribution truncated to $\mathbf{R}+$ and the part in curly brackets describes a discrete mixture for the variable $L$. The behavior of such a perturbation kernel is actually quite simple: it adds a tiny Gaussian noise to $\epsilon^i_{t-1}$ while choosing the new value of $L$ in $\{ L^i_{t-1}-1, L^i_{t-1}, L^i_{t-1}+1\}$ with equal chances.  

*Nota Bene*: the variance chosen for the Gaussian noise has a rather limited effect according to the authors.

In the end, the adaptation of Fearnhead and Taylor's ASMC for HMC could be summarized as following:

<center><br>
For each $i \in 1:N$, sample $h^i_{t} \sim \chi_t(h) \propto \sum_{i=1}^N \tilde{\Lambda}(\tilde{x}^i_{t-2},\hat{x}^i_{t-1}) R(h;h^i_{t-1})$

    
In words, at each time $t$ and for each particle $i$, choose optimal hyperparameters by sampling from the ones that performed the best at time $t-1$ according to the ESJD estimator, then apply a small random perturbation in order to avoid pathological behaviors.

While this method seems overall pretty interesting, it also relies on the hypothesis that parameters that performed well at time $t-1$ are likely to achieve such performances at time $t$, which is not always true. Thus, another strategy is developed in the article.


#### Fearnhead and Taylor Adaptative Sampler implementation

In [ ]:
class FT_Adapt_HSMCSampler(SMCSampler):
    """Implement tuned HMC approach of studied paper."""

    def __init__(self, parameters: ExperimentParameters):
        super().__init__(parameters)
        self.hist_X = []
        self.hist_Z = []
        self.hist_DeltaE = []
        self.hist_epsilon = [stats.uniform(scale = 0.1).rvs(size = self.param.N)]  
        self.hist_L = [np.random.randint(1,101, size = self.param.N)]  

    def _ESJD_estimator(self, x, y, inv_M, DeltaE, L):
        diff_x_y = x - y
        dist_x_y = np.diag(diff_x_y @ inv_M @ diff_x_y.T)
        return dist_x_y * np.minimum(1, np.exp(DeltaE)) / L

    def build_kernel(self, X):
        
        N = self.param.N
        d = d = self.param.d
        
        ####### Matrix M #######
        diag_cov = np.diag(np.cov(X,rowvar=False))
        inv_M = np.diag(diag_cov)
        M = np.diag(np.nan_to_num(1 / diag_cov))
        
        #Handle first move case:
        if len(self.hist_X) == 0:
            
            self._inv_M = inv_M
            new_epsilon = self.hist_epsilon[-1]
            new_L = self.hist_L[-1]
            
            self.hist_X.append(X.copy()) #.copy() may not be required
            
            return M, inv_M, new_epsilon, new_L
            
        
        ####### Epsilon & L #######
        
        #Get old parameters values
        old_X = self.hist_X[-1]
        old_Z = self.hist_Z[-1]
        old_DeltaE = self.hist_DeltaE[-1]
        old_epsilon = self.hist_epsilon[-1]
        old_L = self.hist_L[-1]
        old_inv_M = self._inv_M
        
        #Compute ESJD estimator
        ESJD_estimate = self._ESJD_estimator(old_X, old_Z, old_inv_M, old_DeltaE, old_L)
        ESJD_sample_prob = ESJD_estimate / ESJD_estimate.sum()
        
        self._inv_M = inv_M
        
        #Sample according to good values of ESJD
        ESJD_sample = np.random.choice(N, N,replace = True, p = ESJD_sample_prob)
        
        new_epsilon = np.ones(N)  
        new_L = np.ones(N) 
        
        #Perturbation kernel
            
        new_epsilon = np.abs(stats.multivariate_normal(old_epsilon[ESJD_sample],
                                                       0.015 * np.eye(N)).rvs())
        new_epsilon[new_epsilon>0.7] = 0.7 #condition added to avoid probabilities error
        
        chosen_L = old_L[ESJD_sample]
        new_L = chosen_L + np.random.choice(np.array([- 1, 0, 1]), N)
        new_L[new_L<1] = 1 #avoid L = 0
            

        # Saving values of parameters and X
        self.hist_epsilon.append(new_epsilon.copy())
        self.hist_L.append(new_L.copy())
        self.hist_X.append(X.copy()) #.copy() may not be required
        
        return M, inv_M, new_epsilon, new_L

    def new_leapfrog_integrator(self, X_init, p_init, lambda_t, L,
                                epsilon, inv_M): 
        '''Generic leapfrog intergrator with given parameters).
        X_init, p_init, L and espilon are arrays'''

        X = X_init.copy()
        p = p_init.copy()
        assert np.all(np.abs(inv_M - inv_M.T) < 1e-5), "inv_M is not symmetric !"

        L_copy = L.copy()

        for l in range(np.max(L.astype(int))):
            to_update = (np.maximum(0, L_copy) > 0) * 1
            p += 0.5 * np.einsum('i,ij->ij', to_update, np.einsum('i,ij->ij', 
                                                            epsilon, 
                                                            self.param.grad_log_gamma_t(lambda_t, X)))
            X += np.einsum('i,ij->ij', to_update, np.einsum('i,ij->ij', 
                                                            epsilon,
                                                            np.dot(p, inv_M)))
            p += 0.5 * np.einsum('i,ij->ij', to_update, np.einsum('i,ij->ij', 
                                                            epsilon, 
                                                            self.param.grad_log_gamma_t(lambda_t, X)))
            L_copy -= 1
        return X, p

    def move_through_kernel(self, X, lambda_t):
        """Non-iterated version : single move for each x in X."""
        
        N = self.param.N
        d = self.param.d
        
        # Kernel builds itself before moving
        M, inv_M, epsilon, L = self.build_kernel(X)
        p = stats.multivariate_normal(np.zeros(d), M).rvs(N)
        Z = np.empty((N, d))
        DeltaE = np.ones(N)
        Xbis = X.copy()

        ### New version
        # W plays role of Z
        # p_new_bis plays role of p_new
        Z, p_new_bis = self.new_leapfrog_integrator(X, p, lambda_t, L, epsilon, inv_M)
        log_proba = self.param.log_gamma_t(lambda_t, X) - self.param.log_gamma_t(lambda_t, Z) + \
            (np.einsum('ij,ij->i', p_new_bis, np.dot(p_new_bis, inv_M)) - \
             np.einsum('ij,ij->i', p, np.dot(p, inv_M))) / 2
        
        # Keep new moves or not
        log_acc = np.log(np.random.rand(len(Xbis)))
        keep_it = log_acc < log_proba
        keep_it = keep_it[:, np.newaxis]

        # Xbis is the final X
        X = Z * keep_it + Xbis * (1 - keep_it)

        # Saving values of Z and DeltaE
        self.hist_Z.append(Z)
        self.hist_DeltaE.append(log_proba)
        
        return X

## <div style="padding-left: 15px;"> iii. Another strategy: the pretuning of the kernels

As for Fearnhead and Taylor's adaptative tuning strategy, the main idea of the "pretuning" strategy is to draw pairs $h^i_t = (\epsilon^i_t,L^i_t)$  from a distribution that gives good performing pairs, in the sense of ESJD, greater chances to be sampled. The main difference with the previous strategy relies in the way to build this "performance-rewarding" density. While in Fearnhead and Taylor's approach, previous moves were used to measure hyperparameter's performances, the authors propose to assert these performances by adding a new iterative step in order to build the density from which $(\epsilon^i_t,L^i_t)$ will be sampled.

Assuming that we try to tune hyperparameters at time $t$, the choice of $(\epsilon^i_t,L^i_t)$ can be summarized in **two steps**:

1. For i in N, apply the leapfrog integration with different pairs $(\hat{\epsilon}^i_t,\hat{L}^i_t)$ picked from uniform distributions for each particle. From there, build a new random distribution that will be used to choose the real hyperparameters $(\hat{\epsilon}^i_t,\hat{L}^i_t)$ based on the performances of these N computed steps.
2. For i in N, we apply an HMC step on the same particles as before. However, this time, $(\hat{\epsilon}^i_t,\hat{L}^i_t)$ are generated from the idea distribution built during **step 1**.
</ul>

Several things above require qualification, starting with the choice of the uniform distributions from which trial pairs $(\hat{\epsilon}^i_t,\hat{L}^i_t)$ are sampled from.


### <div style="padding-left: 30px;"> a) Choosing the range of values for $(\hat{\epsilon}, \hat{L})$

As it is described previously for the first stage, we need to pick pairs $(\hat{\epsilon}^i_t,\hat{L}^i_t)$  for each particle $i$.

The value of the step size $\hat{\epsilon}^i_t$ is drawn from a uniform distribution $U[0,\epsilon_{t-1}^*]$ where the initial value $\epsilon_{0}^*$ is set to $0.1$ in the experiments. The next parameters $\epsilon^*$ are then chosen by creating one model for each particle describing the variation of the Hamiltonian, or also the change of energy in function of the step sizes $\epsilon_{t-1}^i$. The model is of the form : 

$$\lvert \Delta E_t^i \rvert = f(\hat{\epsilon}^i_t) + \xi_t^i$$

with $\mathbb{E}[\xi_t^i]=0, \mathbb{Var}[\xi_t^i] = \sigma^2 < \infty$ and $f : {\rm I\!R}^+ \longrightarrow {\rm I\!R}^+$. The authors benefit from result given in a paper written by Betancourt et al. where it is described that taking an $\epsilon^*$ such that $f(\epsilon^*) = |log(0.9)|$ guaranties to get an acceptance rate close to 90 \%. To solve it, the median regression model is chosen over the least squares regression because it is more robust to important fluctuations in the energy which tend to happen when the step size approaches its stability limit.

Proposed values for $\hat{L}^i_t$ are first drawn from a discrete uniform distribution $U[0,L_{\max} ]$. The quantity $L_{\max}$ is initialized by the user. It may be dynamically changed during the algorithm whenever a big amount of values of chosen $\hat{L}^i_t$ are close to $L_{\max}$. In that case, $L_{\max}$ can be dynamically increased by a small value. On the other hand, if chosen $\hat{L}^i_t$ appear to be far from $L_{\max}$, dynamically decreasing is interesting. 

### <div style="padding-left: 30px;"> b) Building the density to sample $(\epsilon, L)$ from

Just as in [II.ii](#FT), the idea is to sample from the $(\hat{\epsilon}^i_t,\hat{L}^i_t)$ that give best performances in terms of estimated ESJD $\tilde{\Lambda}(\tilde{x}^i_{t-1},\hat{x}^i_t)$.  
For that, we simply draw $(\epsilon^i_t,L^i_t)$ in $Cat(w_t^i,\{\hat{\epsilon}^i_t,\hat{L}^i_t\})$, that is to say a categorical distribution where weights $w_t^i \propto \tilde{\Lambda}(\tilde{x}^i_{t-1},\hat{x}^i_t)$. 

Notice that here, pairs $(\epsilon^i_t,L^i_t)$ are chosen directly among  $(\hat{\epsilon}^i_t,\hat{L}^i_t)$ without adding any noise. This may be explained by the fact that the choice of $(\epsilon^i_t,L^i_t)$ with uniform distribution already introduces a form of perturbation that should avoid pathological behaviors.

You can find here an implementation of the above described algorithm

In [ ]:
class Pretuning_HSMCSampler(SMCSampler):
    """Implement tuned HMC approach of studied paper."""

    def __init__(self, parameters: ExperimentParameters):
        super().__init__(parameters)
        self.epsilon_star = 0.1 
        self.Lmax = 100 

    def new_leapfrog_integrator(self, X_init, p_init, lambda_t, L,
                                epsilon, inv_M): 
        '''Generic leapfrog intergrator with given parameters).
        X_init, p_init, L and espilon are arrays'''

        X = X_init.copy()
        p = p_init.copy()
        assert np.all(np.abs(inv_M - inv_M.T) < 1e-5), "inv_M is not symmetric !"

        L_copy = L.copy()

        for l in range(np.max(L.astype(int))):
            to_update = (np.maximum(0, L_copy) > 0) * 1
            p += 0.5 * np.einsum('i,ij->ij', to_update, np.einsum('i,ij->ij', 
                                                            epsilon, 
                                                            self.param.grad_log_gamma_t(lambda_t, X)))
            X += np.einsum('i,ij->ij', to_update, np.einsum('i,ij->ij', 
                                                            epsilon,
                                                            np.dot(p, inv_M)))
            p += 0.5 * np.einsum('i,ij->ij', to_update, np.einsum('i,ij->ij', 
                                                            epsilon, 
                                                            self.param.grad_log_gamma_t(lambda_t, X)))
            L_copy -= 1
        return X, p

    def _ESJD_estimator(self, x, y, inv_M, DeltaE, L):
        diff_x_y = x - y
        dist_x_y = np.diag(diff_x_y @ inv_M @ diff_x_y.T)
        return dist_x_y * np.minimum(1, np.exp(DeltaE)) / L

    def build_kernel(self, X, lambda_t):

        N = self.param.N
        d = self.param.d
        
        #Matrix M
        diag_cov = np.diag(np.cov(X, rowvar=False))
        inv_M = diag_cov * np.eye(d)
        M = np.diag(np.nan_to_num(1 / diag_cov))
        self._inv_M = inv_M

        # Sample epsilon and L
        epsilon = stats.uniform(scale = self.epsilon_star).rvs(size = N)
        L = np.random.randint(1, self.Lmax + 1, size = N)
        
        # Sample p
        p = stats.multivariate_normal(np.zeros(d), M).rvs(N) 

        # Apply the leapfrog integration    
        Z, p_new = self.new_leapfrog_integrator(X, p, lambda_t, L, epsilon, inv_M)

        # Calculate variation of energy
        DeltaE = self.param.log_gamma_t(lambda_t, X) - self.param.log_gamma_t(lambda_t, Z) + \
            (np.einsum('ij,ij->i', p_new, np.dot(p_new, inv_M)) - \
             np.einsum('ij,ij->i', p, np.dot(p, inv_M))) / 2 
          
        # Calculate ESJD
        ESJD_estimate = self._ESJD_estimator(X, Z, inv_M, DeltaE, L)
        ESJD_sample_prob = ESJD_estimate / ESJD_estimate.sum() 

        # Calculate epsilon_star based on quantile regression
        data_reg = pd.DataFrame({'diff_energy':np.abs(DeltaE),
                                 'eps':epsilon**2})
        model = smf.quantreg('diff_energy ~ eps', data_reg)
        res = model.fit(q=.5) 

        quotien = (np.abs(np.log(0.9)) - res.params['Intercept']) / res.params['eps']
        if (quotien>0):
          self.epsilon_star = np.sqrt(quotien)

        # Sample epsilon and L
        ESJD_sample = np.random.choice(N, N, replace = True, p = ESJD_sample_prob)
        new_epsilon = epsilon[ESJD_sample]
        new_L = L[ESJD_sample]

        # Update Lmax
        if ((100*np.sum(new_L>self.Lmax-5)/len(new_L))>90):
            self.Lmax += 5
        elif ((100*np.sum(new_L>self.Lmax-5)/len(new_L))<10):
            self.Lmax -= 5
        
        return M, inv_M, new_epsilon, new_L

    def move_through_kernel(self, X, lambda_t):
        """Non-iterated version : single move for each x in X."""

        N = self.param.N
        d = self.param.d
        
        # Kernel builds itself before moving
        M, inv_M, epsilon, L = self.build_kernel(X, lambda_t)

        # Sample p
        p = stats.multivariate_normal(np.zeros(d), M).rvs(N)

        # Apply the leapfrog integration
        Z, p_new = self.new_leapfrog_integrator(X, p, lambda_t, L, epsilon, inv_M)
        log_proba = self.param.log_gamma_t(lambda_t, X) - self.param.log_gamma_t(lambda_t, Z) + \
            (np.einsum('ij,ij->i', p_new, np.dot(p_new, inv_M)) - \
             np.einsum('ij,ij->i', p, np.dot(p, inv_M))) / 2
        
        # Keep new moves or not
        log_acc = np.log(np.random.rand(len(X)))
        keep_it = log_acc < log_proba
        keep_it = keep_it[:, np.newaxis]

        X = Z * keep_it + X * (1 - keep_it)
        
        return X

# III. Numerical Applications

In this section, we want to emphasize the importance of adapting SMC samplers and consequently tuning their Markov kernels. We want also to emphasize that HMC Kernels within SMC seem to perform very well in high dimension, particularly when they are tuned. The main purpose of the experiment is to compare adaptive and non-adaptive version of SMC samplers over different dimensions. The first adaptive method uses the Fearnhead and Taylor approach for HMC Kernels and corresponds to the Class **FT_Adapt\_HMCSampler** in the code ; the second adaptive method is the pre-tuning approach also with HMC Kernels, and corresponds to the Class **Pretuning_HMCSampler**.  We also include an SMC sampler with MALA kernels, another one with tuned Independence Sampler steps, and a last one with non tuned HMC kernels.

The number of particles $N$ was set to $1000$ and the resampling is done when the ESS drops below $\frac N2$. Adaptative HMC-based samplers are initialized with uniform random draws of $\epsilon$ on $[0,0.1]$ and L on $[1,100]$. Non-tuned HMC is initialized with $\epsilon = 0.05$ and $L = 5$ and MALA sampler is set with a step-size $\sigma = d^{-1/3}$, where $d$ is the dimension. 

The experiment we chose corresponds to the first experiment in the original paper. Indeed, here we consider a tempering sequence that starts from an isotropic Gaussian $\pi_0 = \mathcal{N}(0_d,I_d)$ and end at a shifted and correlated Gaussian $\pi_T = \mathcal{N}(\mu,\Sigma)$, where $\mu = 2*I_d$. For the covariance we set the off-diagonal correlation to $0.7$ and the marginal variances to elements of the equally spaced sequence $\widetilde{\Sigma} = [0.1,...,10]$.

Hence we get the covariance matrix :

$$\Sigma = diag\widetilde{\Sigma}^{1/2} corr(X) diag \widetilde{\Sigma}^{1/2}$$

Here is a visual example of where we start as an initial distribution and where we end as a target distribution.

In [ ]:
d = 2
correlation = 0.7*np.ones((d, d))
np.fill_diagonal(correlation, 1)
diag_var = np.linspace(start=0.1, stop=10, num=d)
theta_d = np.dot(np.diag(diag_var**0.5), correlation).dot(np.diag(diag_var**0.5))
inv_theta = inv(theta_d)

dist1 = pd.DataFrame(stats.multivariate_normal(np.zeros(d), np.eye(d)).rvs(10000))
dist2 = pd.DataFrame(stats.multivariate_normal(2 * np.ones(d), theta_d).rvs(10000))
dist1['Distribution'] = 'Initial' 
dist2['Distribution'] = 'Target' 
dist = pd.concat([dist1,dist2]).reset_index(drop=True)

ax = sns.displot(dist,x=dist.iloc[:,0],y=dist.iloc[:,1],kind='kde',hue='Distribution')
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.title("Example in 2D - Experiment One")
plt.show()

This example is well suited to observe the behavior of the samplers with respect to changes in the variance scale, the correlation and the shifted mean of the target.

In [ ]:
def experiment1(dimension, particles=1000, alpha=0.5):
    N = particles
    d = dimension
    Id = np.eye(d)
    correlation = 0.7*np.ones((d, d))
    np.fill_diagonal(correlation, 1)
    diag_var = np.linspace(start=0.1, stop=10, num=d)
    theta_d = np.dot(np.diag(diag_var**0.5), correlation).dot(np.diag(diag_var**0.5))
    inv_theta = inv(theta_d)
    normalizing_constant = 1 / ((2 * np.pi) ** (d / 2) * 
                                np.sqrt(np.linalg.det(theta_d)))
    grad_log_gamma_t = lambda lambda_t, X : (lambda_t - 1) * X - \
        lambda_t * np.dot(inv_theta, (X - 2).T).T
    _experiment1 = ExperimentParameters(
        log_prior=lambda X: - 0.5 * np.einsum('ij,ij->i', X, X) - \
           np.log((2 * np.pi) ** (d / 2) * np.sqrt(np.linalg.det(Id))),
        log_likelihood=lambda X: 0.5 * np.einsum('ij,ij->i', X, X) - \
            0.5 * np.einsum('ij,ij->i', X - 2 * np.ones(X.shape),
                            np.dot(inv_theta, (X - 2 * np.ones(X.shape)).T).T) \
            + np.log((2 * np.pi) ** (d / 2) * np.sqrt(np.linalg.det(Id))) - \
            np.log((2 * np.pi) ** (d / 2) * np.sqrt(np.linalg.det(theta_d))),
        pi0=stats.multivariate_normal(np.zeros(d), Id),
        N=N, d=d, alpha=alpha, normalizing_constant=normalizing_constant, 
        grad_log_gamma_t=grad_log_gamma_t
    )
    return _experiment1

Let's now define a class that will be useful to do some statistics about the performance of our algorithms.

In [ ]:
class StatsSMC:

    def __init__(self, sampler : SMCSampler):
        """Argument : sampler -- uninstantiated class of SMCSampler."""
        self.sampler = sampler

    def MSE_mean_first_component(self, true_mean, parameters, repetitions):
        """Parameters -- class instantialised"""
        MSE_mean = np.empty(repetitions)
        for r in range(repetitions):
            if r % 10 == 0:
                print('repetition :', r)

            # run sampler
            sampler = self.sampler(parameters)
            sampler.run()
            X = sampler.X

            # compute MSEs
            mean = np.mean(X[:, 0])
            MSE_mean[r] = np.mean((mean - true_mean)**2)

        return MSE_mean

    def MSE_over_dimensions(self, true_mean, parameters_function, 
                            repetitions, dimensions):
        """parameters_functions -- like experiment1"""
        results = {}
        for d in dimensions:
            print(d)
            parameters = parameters_function(dimension=d)
            results[d] = self.MSE_mean_first_component(true_mean,
                                                       parameters, repetitions)

        return results

    def get_lambdas(self, repetitions, parameters_function, dimension):
        results = []
        for i in range(repetitions):
            sampler = self.sampler(parameters_function(dimension=dimension))
            sampler.run()
            results.append(sampler.lambdas)
        return results

    def evol_lambdas(self, repetitions, parameters_function, dimension):
        result_lambas = self.get_lambdas(repetitions, parameters_function, dimension)
        length = max(map(len, result_lambas))
        evol = np.array([xi+[1]*(length-len(xi)) for xi in result_lambas])
        return np.mean(evol, axis=0)

    def boxplot_first_component(self, repetitions, parameters_function, dimension):
        results = []
        for i in range(repetitions):
            sampler = self.sampler(parameters_function(dimension=dimension))
            sampler.run()
            results.append(sampler.X[:, 0])
        return results

The following cells run 40 experiments with 1000 particles for each of the dimensions 5, 10, 50, 100, 150 and 200.

SMC with MALA kernel takes 1 minute to do all the computations. SMC with non tuned HMC and SMC with Tuned Independence Sampler kernel take 3 minutes to do the same ; SMC with tuned HMC kernels take 1 hour for Fearnhead and Taylor and 1h20 for Pretuning.

In [ ]:
%%time 
mse_mala_first_dim = StatsSMC(MALASMCSampler).MSE_over_dimensions(true_mean=2,
                                                   parameters_function=experiment1, 
                                                   dimensions=[5, 10, 50, 100, 150, 200], 
                                                   repetitions=40)

In [ ]:
%%time 
mse_tuned_ismc_first_dim = StatsSMC(TunedISMC).MSE_over_dimensions(true_mean=2,
                                                   parameters_function=experiment1, 
                                                   dimensions=[5, 10, 50, 100, 150, 200], 
                                                   repetitions=40)

In [ ]:
%%time 
mse_non_tuned_hsmc_first_dim = StatsSMC(NonTunedHSMCSampler).MSE_over_dimensions(true_mean=2,
                                                   parameters_function=experiment1, 
                                                   dimensions=[5, 10, 50, 100, 150, 200], 
                                                   repetitions=40)

In [ ]:
%%time 
mse_ft_first_dim = StatsSMC(FT_Adapt_HSMCSampler).MSE_over_dimensions(true_mean=2,
                                                   parameters_function=experiment1, 
                                                   dimensions=[5, 10, 50, 100, 150, 200], 
                                                   repetitions=40)

In [ ]:
%%time 
mse_pt_first_dim = StatsSMC(Pretuning_HSMCSampler).MSE_over_dimensions(true_mean=2,
                                                             parameters_function=experiment1, 
                                                             dimensions=[5, 10, 50, 100, 150, 200], 
                                                             repetitions=40)

In [ ]:
my_exp1_mean = pd.DataFrame([{d:np.mean(e) for d, e in mse_pt_first_dim.items()}.values(),
                             {d:np.mean(e) for d, e in mse_ft_first_dim.items()}.values(),
                             {d:np.mean(e) for d, e in mse_non_tuned_hsmc_first_dim.items()}.values(),
                             {d:np.mean(e) for d, e in mse_tuned_ismc_first_dim.items()}.values(), 
                             {d:np.mean(e) for d, e in mse_mala_first_dim.items()}.values()], 
                            index=['Pretuning', 'FearnheadTaylor', 'NonTunedHMC', 'TunedISMC', 'MALA'], 
                            columns=[5, 10, 50, 100, 150, 200]).T
my_exp1_var = pd.DataFrame([{d:np.var(e) for d, e in mse_pt_first_dim.items()}.values(),
                             {d:np.var(e) for d, e in mse_ft_first_dim.items()}.values(),
                             {d:np.var(e) for d, e in mse_non_tuned_hsmc_first_dim.items()}.values(),
                             {d:np.var(e) for d, e in mse_tuned_ismc_first_dim.items()}.values(), 
                             {d:np.var(e) for d, e in mse_mala_first_dim.items()}.values()], 
                            index=['Pretuning', 'FearnheadTaylor', 'NonTunedHMC', 'TunedISMC', 'MALA'], 
                            columns=[5, 10, 50, 100, 150, 200]).T

We first present our results concerning the MSE of the mean of the first component for each dimension. 

As we can see, Pretuning seems to be the most efficient sampler, even in high dimensions. Fearnhead and Taylor seems to be the second best performing one, while Non-Tuned HMC and Tuned IS within SMC seem to perform relatively poorly in comparison.

*Note*: we did not include SMC with MALA kernel here because we are not satisfied with the result, and we think that it can be dued to something that we do not know about some properties of MALA in high dimensions, particularly regarding the step-size $\sigma$.

In [ ]:
my_exp1_mean[['Pretuning', 'FearnheadTaylor', 'NonTunedHMC', 'TunedISMC']].plot(title='Empirical MSE');

We can see here that SMC with MALA kernel is realy under-performing in comparison to other procedures.

In [ ]:
my_exp1_mean.plot(title='Empirical MSE');

To investigate the difference between the performances, we have studied the temperature through which each algorithm passes in dimension 100. (We do a mean on 10 experiment in dimension 100 with 1000 particles each).

In [ ]:
lambdas_ismc = StatsSMC(TunedISMC).evol_lambdas(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
lambdas_pt = StatsSMC(Pretuning_HSMCSampler).evol_lambdas(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
lambdas_ft = StatsSMC(FT_Adapt_HSMCSampler).evol_lambdas(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
lambdas_hmc = StatsSMC(NonTunedHSMCSampler).evol_lambdas(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
lambdas_mala = StatsSMC(MALASMCSampler).evol_lambdas(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
my_dict = {'ISMC': lambdas_ismc, 'Pretuning': lambdas_pt, 'FT': lambdas_ft,
           'HMC': lambdas_hmc, 'MALA': lambdas_mala}

As we can see, non tuned HMC and MALA go faster to the final density. It is interesting to note that the procedures that converge fast to the final density are the under-performing ones, what is quite intuitive.

In [ ]:
with open('lambdas', 'rb') as f:
    my_dict = pickle.load(f)
pd.DataFrame.from_dict(my_dict, orient='index').fillna(1).T.plot(title="Evolution of the temperature (dimension 100)");

We now give a more illustrated example of the distribution obtained with each of the procedure in dimension 100, and compare it to the expected distribution.

In [ ]:
first_compo_ismc = StatsSMC(TunedISMC).boxplot_first_component(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
first_compo_pt = StatsSMC(Pretuning_HSMCSampler).boxplot_first_component(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
first_compo_ft = StatsSMC(FT_Adapt_HSMCSampler).boxplot_first_component(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
first_compo_hmc = StatsSMC(NonTunedHSMCSampler).boxplot_first_component(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
first_compo_mala = StatsSMC(MALASMCSampler).boxplot_first_component(repetitions=10,
                                                parameters_function=experiment1, 
                                                dimension=100)
my_dict_first_compo = {'ISMC': first_compo_ismc, 'Pretuning': first_compo_pt, 'FT': first_compo_ft,
           'HMC': first_compo_hmc, 'MALA': first_compo_mala}
first_compo_df = pd.DataFrame.from_dict({ind: np.array(v).flatten() for ind,v in my_dict_first_compo.items()})

In [ ]:
# add true distribution
d = 100
correlation = 0.7*np.ones((d, d))
np.fill_diagonal(correlation, 1)
diag_var = np.linspace(start=0.1, stop=10, num=d)
theta_d = np.dot(np.diag(diag_var**0.5), correlation).dot(np.diag(diag_var**0.5))


first_compo_df['Target'] = stats.multivariate_normal(2 * np.ones(d), theta_d).rvs(10000)[:, 0]

We can see here that the procedures with the smallest MSE concerning their mean are again the best performing ones (they could have been for example with a much smaller variance than the others, which is not the case).

Nonetheless, we can see that all procedures tend to have a smaller variance than what was expected, and that they are all biased towards the initial distribution.

In [ ]:
first_compo_df.boxplot(column=['ISMC', 'HMC', 'FT', 'Pretuning', 'Target'], grid=False).set_title('First dimension of drawn particles (dimension 100)');

*Note* :

We haven't compared the procedures in terms of computation cost, because our code can surely be optimized. For instance, the difference between the run time of our implementation of MALA and Pretuning is too big to be explained by their theoretical construction. 

# Conclusion



In the paper that we studied, and in the experiments that we made, we can see that HMC kernel are well suited for SMC procedures even in a high-dimensional setting. Nonetheless, it seems crucial to tune those kernels at each step.

The tuning of those kernels can be made with two approaches : the one of Fearnhead and Taylor, and the Pretuning one described in the studied paper. Even if those procedures can be much slower than the samplers without tuning, they increase the performance in such a manner that their use is recommended, in particular in high dimensions.

# References
<div style="padding-left: 15px;"><br>

**[1]** Buchholz, A., Chopin, N., & Jacob, P. E. (2020). *Adaptive tuning of Hamiltonian Monte Carlo within sequential Monte Carlo.* Bayesian Analysis.

**[2]** Chopin, Nicolas. (2001). *A Sequential Particle Filter Method for Static Models* Biometrika. 89. 10.1093/biomet/89.3.539. 

**[3]** Fearnhead, Paul; Taylor, Benjamin M. (2013) *An Adaptive Sequential Monte Carlo Sampler.* Bayesian Anal. 8 , no. 2, 411--438.
<a name="bib">